In [2]:
#System Imports
import os
import glob
import dill as pickle



#Math Imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import sklearn as sk



#Data Analysis Imports
import scipy.io
import librosa
import librosa.display
import pandas as pd
from pandas import DataFrame



#NN Import

import tensorflow as tf

import utils as util

util_class = util.utils_cls()

tf.reset_default_graph()

In [3]:
def get_directories(ROOT_PATH, directory):

    directories = [d for d in os.listdir(directory)
                if os.path.isdir(os.path.join(directory, d))]

    return directories

In [6]:

def get_data():
    
    if os.name == 'nt':
        ROOT_PATH = "D:/Masters/Data"
    else:
        ROOT_PATH = "/home/tim/Documents/Masters/Data"
        
    train_data_directory = os.path.join(ROOT_PATH, "Autoencoder test/Training")
    test_data_directory = os.path.join(ROOT_PATH, "Autoencoder test/Testing")
    print('train data directory' + str(train_data_directory))
    print('test data directory' + str(test_data_directory))  
    train_directories = get_directories(ROOT_PATH, train_data_directory)
    test_directories = get_directories(ROOT_PATH, test_data_directory)
    print('train  directories' + str(train_directories))

    raw_sounds_tr, sr_tr, tr_labels = util_class.load_sound_wave(parent_dir=train_data_directory, sub_dirs=train_directories, file_ext="*.wav")
    raw_sounds_ts, sr_ts, ts_labels = util_class.load_sound_wave(parent_dir=test_data_directory, sub_dirs=test_directories, file_ext="*.wav")
    return raw_sounds_tr, raw_sounds_ts, sr_tr, sr_ts, tr_labels, ts_labels

In [7]:

raw_sounds_tr, raw_sounds_ts, sr_tr, sr_ts, tr_labels, ts_labels = get_data()

train data directoryD:/Masters/Data\Autoencoder test/Training
test data directoryD:/Masters/Data\Autoencoder test/Testing
train  directories['Noise', 'Minke']


<class 'numpy.ndarray'>


<class 'numpy.ndarray'>


In [32]:
str_raw_sounds_tr = str(raw_sounds_tr)
str_raw_sounds_ts = str(raw_sounds_ts)
print(len(raw_sounds_tr))

print(str_raw_sounds_ts)

897921
[-0.01684551 -0.01755317 -0.01826035 ..., -0.00663429 -0.00532464  0.        ]


In [9]:
if os.name == 'nt':
    training_file = "D:\Masters\Data/Autoencoder test/Training/training_tester.pkl"
    testing_file = "D:\Masters\Data/Autoencoder test/Testing/testing_tester.pkl"
else:
    
    training_file = "/home/tim/Documents/Masters/Data/Autoencoder test/Training/training_tester.pkl"
    testing_file = "/home/tim/Documents/Masters/Data/Autoencoder test/Testing/testing_tester.pkl"
        
util_class.DILL_pickle_files(raw_sounds_tr, training_file)

util_class.DILL_pickle_files(raw_sounds_ts, testing_file)





In [10]:
pickled_sounds_tr = util_class.DILL_unpickle_files(training_file)

pickled_sounds_ts = util_class.DILL_unpickle_files(testing_file)

In [11]:

sounds_tr = tf.convert_to_tensor(pickled_sounds_tr, dtype=tf.float32, name='sounds_tr' )
sounds_ts = tf.convert_to_tensor(pickled_sounds_ts, dtype=tf.float32, name='sounds_ts')


print(sounds_tr)

Tensor("sounds_tr:0", shape=(897921,), dtype=float32)


In [12]:
    def one_hot_encode( labels):
        n_labels = labels
        n_unique_labels = len(np.unique(labels))
        one_hot_encode = np.eye(n_unique_labels)

        return one_hot_encode


In [69]:

def RNN(x, number_of_layers, n_hidden, labels):
    
    init_state = tf.placeholder(tf.float32, [number_of_layers, 2, batch_size, n_hidden])

    state_per_layer_list = tf.unstack(init_state, axis=0)
    #rnn_tuple_state = tuple([tf.contrib.rnn.LSTMStateTuple(state_per_layer_list[idx][0], state_per_layer_list[idx][1]) for idx in range(number_of_layers)])
    
    cell = tf.contrib.rnn.LSTMCell(n_hidden, state_is_tuple=True)
    
    stacked_lstm = tf.contrib.rnn.MultiRNNCell([cell] * number_of_layers, state_is_tuple=True)
    initial_state = tf.identity(stacked_lstm.zero_state(batch_size, tf.float32), name = 'initial_state')
    
    Outputs, final_state = tf.nn.dynamic_rnn(stacked_lstm, x, dtype = tf.float32)
    final_state = tf.identity(final_state, name='final_state')
   
    Logits = tf.contrib.layers.fully_connected(Outputs, 
                                               num_outputs = len(labels),
                                               activation_fn = None,
                                              weights_initializer = (tf.truncated_normal_initializer(stddev = 0.1)),
                                              biases_initializer = (tf.zeros_initializer()))
    return (Logits, final_state, initial_state)
    
   

In [70]:

def get_batches(sounds, batch_size, seq_length, labels):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function

    n_batches = int(len(sounds) / (batch_size * seq_length))

    # Drop the last few characters to make only full batches
    xdata = np.array(sounds[: n_batches * batch_size * seq_length])
    ydata = np.array(labels[1: n_batches * batch_size * seq_length + 1])

    x_batches = np.split(xdata.reshape(batch_size, -1), n_batches, 1)
    y_batches = np.split(ydata.reshape(batch_size, -1), n_batches, 1)

    return np.asarray(list(zip(x_batches, y_batches)))


In [71]:
#def main():
"""

:return:
"""
tf.reset_default_graph()

#sounds_ts = tf.convert_to_tensor(raw_sounds_ts, dtype=tf.float32, name='sounds_ts')
######################################
tr_labels = one_hot_encode(tr_labels)
ts_labels = one_hot_encode(ts_labels)



lr = 0.01
num_epochs = 1000
batch_size = 50
display_step = 200

# Network Parameters

number_of_layers = 2

n_hidden = 300
n_classes = 2

x = tf.placeholder(tf.float32, [None, 300, 1], name='x')
y = tf.placeholder(tf.float32, [None, n_classes], name='y')

weight = tf.Variable(tf.random_normal([n_hidden, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))
#raw_sounds_tr = tf.unstack(raw_sounds_tr)
#raw_sounds_ts = tf.unstack(raw_sounds_ts)

Logits, final_state, initial_state = RNN(x, number_of_layers, n_hidden, tr_labels)

# Define loss ///and optimizer
cost = -tf.reduce_sum(y * tf.log(Logits))


optimizer = tf.train.AdamOptimizer(lr).minimize(cost)

    # Gradient Clipping
gradients = optimizer.compute_gradients(cost)
capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
train_op = optimizer.apply_gradients(capped_gradients)
# Evaluate model
correct_pred = tf.equal(tf.argmax(Logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()



batches_tr = get_batches(sounds_tr, batch_size, display_step, tr_labels)
batches_ts = get_batches(sounds_ts, batch_size, display_step, ts_labels)



with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {x: batches_tr[0][0]})

        for batch_i, (x, y) in enumerate(batches_tr):
            feed = {
                x: batches_tr,
                y: tr_labels,
                initial_state: state,
                lr: lr}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches_tr) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches_tr),
                    train_loss))

    # Save Model
    #saver = tf.train.Saver()
    #saver.save(sess, save_dir)
    print('Model Trained and Saved')
    # for itr in range(training_iters):
    #     offset = (itr * batch_size) % (tr_labels.shape[0] - batch_size)
    #     batch_x = raw_sounds_tr
    #     print(batch_x)
    #     batch_x = np.squeeze(batch_x)
    #     batch_y = tr_labels[offset:(offset + batch_size)]
    #     _, c = session.run([optimizer, loss_f], feed_dict={x: batch_x, y: batch_y})
    # 
    #     if itr % display_step == 0:
    #         # Calculate batch accuracy
    #         acc = session.run(accuracy, feed_dict={x: batch_x, y: batch_y})
    #         # Calculate batch loss
    #         loss = session.run(loss_f, feed_dict={x: batch_x, y: batch_y})
    #         print("Iter " + str(epoch) + ", Minibatch Loss= " + \
    #               "{:.6f}".format(loss) + ", Training Accuracy= " + \
    #               "{:.5f}".format(acc))
    # 
    # print('Test accuracy: ', round(session.run(accuracy, feed_dict={x: batch_x, y: ts_labels}), 3))
    # 


ValueError: Dimensions must be equal, but are 600 and 301 for 'rnn/while/rnn/multi_rnn_cell/cell_0/cell_0/lstm_cell/MatMul_1' (op: 'MatMul') with input shapes: [?,600], [301,1200].